# Thermal simulation of a room using `dm4bem`
This notebook explains in detail a thermal model simulated in Python using the `dm4bem` library.
## 1. Importing libraries
We start by importing the necessary libraries:

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
import dm4bem

## 2. Define the geometric dimensions of the room
Define the dimensions of the room (length, width, height), the surfaces of the walls, floor and ceiling, and the interior volume.

In [2]:
L, L2, H = 2.5, 2, 3
S_wall1 = L * H
S_wall2 = L2 * H
S_wall = S_wall1 + S_wall2
S_floor = S_roof = L**2
V_air = L**2 * H

## 3. Solar absorption data
Solar irradiance and absorption coefficients for different surfaces.

In [3]:
E = 300  # Irradiation solaire (W/m²)
alphawout = 0.25
alphawindow = 0.38
alpharoof = alphafloor = 0.2

## 4. Fixed temperatures
Indoor and outdoor reference temperatures for calculations ## 3. Solar absorption data
Solar irradiance and absorption coefficients for different surfaces.

In [4]:
To, Ti = 10, 22

## 5. Defining material properties
Each material is characterized by its thermal conductivity, density, specific heat, thickness and associated surface area.

In [5]:
air = {'Density': 1.2, 'Specific heat': 1000}
concrete = {'Conductivity': 1.4, 'Density': 2300, 'Specific heat': 880, 'Width': 0.24, 'Surface': 3 * S_wall}
insulation = {'Conductivity': 0.027, 'Density': 55.0, 'Specific heat': 1210, 'Width': 0.06, 'Surface': 3 * S_wall}
glass = {'Conductivity': 1.4, 'Density': 2500, 'Specific heat': 1210, 'Width': 0.04, 'Surface': S_wall}
floor = {'Conductivity': 0.15, 'Density': 700, 'Specific heat': 1500, 'Width': 0.02, 'Surface': S_floor}
roof = {'Conductivity': 0.25, 'Density': 850, 'Specific heat': 1090, 'Width': 0.013, 'Surface': S_roof}

wall = pd.DataFrame.from_dict({'Concrete': concrete, 'Insulation': insulation, 'Glass': glass, 'Floor': floor, 'Roof': roof}, orient='index')

## 6. Radiative coefficients & calculation of LW exchange
Calculations of conductances related to thermal radiation inside the room.

In [18]:
ε_wLW, ε_gLW = 0.85, 0.90
σ = 5.67e-8
Tm = 273 + 10
Fwg = glass['Surface'] / concrete['Surface']
GLW1 = 4 * σ * Tm**3 * ε_wLW / (1 - ε_wLW) * concrete['Surface']
GLW12 = 4 * σ * Tm**3 * Fwg * concrete['Surface']
GLW2 = 4 * σ * Tm**3 * ε_gLW / (1 - ε_gLW) * glass['Surface']
GLW = 1 / (1 / GLW1 + 1 / GLW12 + 1 / GLW2)

hi, ho = 8., 25.
λ_c, w_c = concrete['Conductivity'], concrete['Width']
λ_i, w_i = insulation['Conductivity'], insulation['Width']
λ_g, w_g = glass['Conductivity'], glass['Width']
λ_f, w_f = floor['Conductivity'], floor['Width']
λ_r, w_r = roof['Conductivity'], roof['Width']
S1, S2, S3 = S_wall1, S_wall2, S_wall1
S_floor = floor['Surface']
S_roof = roof['Surface']
S_door = S1-S2
λ_door = 0.22
w_door = 0.04
S_wind = glass['Surface']

## 7. Ventilation exchange
Air exchange is modeled with an ACH hourly exchange rate.

In [19]:
ACH = 1
Va_dot = ACH / 3600 * V_air
Gv = air['Density'] * air['Specific heat'] * Va_dot

## 8. Total heat capacity
Calculation of heat capacity (stored heat) in materials and air.

In [20]:
C = wall['Density'] * wall['Specific heat'] * wall['Surface'] * wall['Width']
C['Air'] = air['Density'] * air['Specific heat'] * V_air

# Building the thermal model
In this section, we continue building the thermal model with :
- Definition of the incidence matrix A
- Calculation of conductances G
- Definition of nodal heat capacities C
- Temperature (b) and flux (f) sources
- Thermal model output## 8. Total heat capacity
Calculation of heat capacity (stored heat) in materials and air.

In [21]:
# Définition des noms de nœuds de température et flux
θ = [f'θ{i}' for i in range(19)]
q = [f'q{i}' for i in range(29)]


## Creating the A incidence matrix
The A matrix represents the thermal topology of the system, linking temperature nodes via thermal conductance elements. It is filled in manually here for each thermal link present in the building.

In [22]:
A = np.zeros([29, 19])
# Remplissage manuel de la matrice A
A[0, 0] = 1
A[1, 0] = -1; A[1, 1] = 1
A[2, 1] = -1; A[2, 2] = 1
A[3, 3] = 1
A[4, 3] = -1; A[4, 4] = 1
A[5, 4] = -1; A[5, 5] = 1
A[6, 5] = -1; A[6, 6] = 1
A[7, 6] = -1; A[7, 2] = 1
A[8, 7] = 1
A[9, 7] = -1; A[9, 8] = 1
A[10, 8] = -1; A[10, 9] = 1
A[11, 9] = -1; A[11, 2] = 1
A[12, 9] = -1; A[12, 6] = 1
A[13, 9] = -1; A[13, 1] = 1
A[14, 10] = 1
A[15, 10] = -1; A[15, 11] = 1
A[16, 11] = -1; A[16, 12] = 1
A[17, 12] = -1; A[17, 2] = 1
A[18, 12] = -1; A[18, 6] = 1
A[19, 12] = -1; A[19, 1] = 1
A[20, 2] = 1
A[21, 13] = 1
A[22, 13] = -1; A[22, 14] = 1
A[23, 14] = -1; A[23, 15] = 1
A[24, 15] = -1; A[24, 2] = 1
A[25, 16] = 1
A[26, 16] = -1; A[26, 17] = 1
A[27, 17] = -1; A[27, 18] = 1
A[28, 18] = -1; A[28, 2] = 1

## Calculation of thermal conductances G
Each heat flow (line in A) is associated with a conductance G, which depends on the nature of the material and the surface crossed.
We assemble all the Gs into a vector, which is then converted into a diagonal matrix for matrix calculation.

In [23]:
# Variables nécessaires définies précédemment (λ, w, surfaces...)
# Calcul manuel des conductances comme dans le script initial
# Voir code précédent pour la formulation complète...
G = np.hstack([
    hi * S_wind,
    (λ_g / (2 * w_g)) * S_wind,
    ho * S_wind,
    hi * S1, (λ_c / (2 * w_c)) * S1, (λ_i / w_i) * S1, (λ_c / (2 * w_c)) * S1, ho * S1,
    hi * S2, (λ_c / (2 * w_c)) * S2, (λ_i / w_i) * S2, ho * S2,
    GLW, GLW, GLW, GLW,
    hi * S3, (λ_c / (2 * w_c)) * S3, (λ_i / w_i) * S3, ho * S3,
    (2 * hi + λ_door / w_door) * S_door,
    hi * S_floor, (λ_f / (2 * w_f)) * S_floor, (λ_f / (2 * w_f)) * S_floor, ho * S_floor,
    hi * S_roof, (λ_r / (2 * w_r)) * S_roof, (λ_r / (2 * w_r)) * S_roof, ho * S_roof
])

# Thermal modeling of a part using `dm4bem`.
## Definition of nodal thermal capacities

Here we define the thermal capacities associated with each node. You can choose whether or not to include air and glass in the calculations.

This influences system dynamics (thermal accumulation).

In [24]:

# Capacité thermique par nœud 
neglect_air_glass = True

if neglect_air_glass:
    C = np.array([0, concrete['Density'] * concrete['Specific heat'] * concrete['Surface'] * concrete['Width'], 0,
                  insulation['Density'] * insulation['Specific heat'] * insulation['Surface'] * insulation['Width'],
                  0, 0, 0, 0 ,0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
else:
    C = np.array([0, concrete['Density'] * concrete['Specific heat'] * concrete['Surface'] * concrete['Width'], 0,
                  insulation['Density'] * insulation['Specific heat'] * insulation['Surface'] * insulation['Width'],
                  0, 0, glass['Density'] * glass['Specific heat'] * glass['Surface'] * glass['Width'], 0, 0, 0,
                  0, 0, air['Density'] * air['Specific heat'] * V_air, 0, 0, 0, 0, 0, 0])
pd.DataFrame(C, index=θ)


,0
θ0,0.0
θ1,19673280.0
θ2,0.0
θ3,161716.5
θ4,0.0
θ5,0.0
θ6,0.0
θ7,0.0
θ8,0.0
θ9,0.0



## Definition of temperature and flow sources

- b` is the vector of temperature sources (e.g. imposed outdoor and indoor temperatures).
- f` is the vector of heat flux sources (e.g. solar or internal heat gain).

In [25]:

b = pd.Series(['To', 0, 0, 'To', 0, 0, 0, 0, 'Ti', 0, 0, 0, 0, 0, 'Ti', 0, 0, 0, 0, 0, 'Ti', 'Ti',0,0,0,'Ti',0,0,0],
              index=q)

f = pd.Series(['Φ1', 0, 0, 'Φ2', 0, 0, 0, 0, 0, 0, 0, 0, 0, 'Φ3', 0, 0, 'Φ4', 0, 0],
              index=θ)



## Selecting the temperature node of interest

Here we're interested in the indoor air temperature (node `θ2`).

In [26]:

y = np.zeros(19)           
y[[2]] = 1               
pd.DataFrame(y, index=θ)


,0
θ0,0.0
θ1,0.0
θ2,1.0
θ3,0.0
θ4,0.0
θ5,0.0
θ6,0.0
θ7,0.0
θ8,0.0
θ9,0.0




## Creating the complete thermal model

Gather all elements in a dictionary to prepare for steady-state simulation or analysis.

In [27]:

A = pd.DataFrame(A, index=q, columns=θ)
G = pd.Series(G, index=q)
C = pd.Series(C, index=θ)
b = pd.Series(b, index=q)
f = pd.Series(f, index=θ)
y = pd.Series(y, index=θ)

TC = {"A": A, "G": G, "C": C, "b": b, "f": f, "y": y}



## Steady-state simulation

The steady-state thermal system is solved by inverting the linear system `θss`.

In [28]:

A_np = A.values
G_mat = np.diag(G.values)
bss = np.zeros(29)
bss[[0, 3]] = 10
bss[[8, 14, 20, 21, 25]] = 22

fss = np.zeros(19)
diag_G = pd.DataFrame(np.diag(G), index=G.index, columns=G.index)
θss = np.linalg.inv(A.T @ diag_G @ A) @ (A.T @ diag_G @ bss + fss)

print(f'θss = {np.around(θss, 2)} °C')


θss = [13.94 15.74 16.54 10.27 10.99 15.71 16.44 21.74 21.02 16.38 20.11 18.22
 16.36 21.02 18.94 16.85 20.17 18.65 17.12] °C



## System state representation

The thermal model is transformed into a system of state equations using the `tc2ss` function from `dm4bem`. This makes it easy to simulate dynamic behavior.

In [29]:

[As, Bs, Cs, Ds, us] = dm4bem.tc2ss(TC)

bT = np.array([10, 10, 22, 22, 22, 22, 22])  # [To, To, Ti, Ti, Ti, Ti, Ti]
fQ = np.array([0, 0, 0, 0])                  # [Φ1, Φ2, Φ3, Φ4]
uss = np.hstack([bT, fQ]) 

inv_As = pd.DataFrame(np.linalg.inv(As),
                      columns=As.index, index=As.index)

yss = (-Cs @ inv_As @ Bs + Ds) @ uss
yss = float(yss.values[0])

print(f'yss_θ3 = {yss:.2f} °C')
print(f'Error between DAE and state-space: {abs(θss[2] - yss):.2e} °C')


yss_θ3 = 16.54 °C
Error between DAE and state-space: 1.42e-14 °C
